In [4]:
# Install required packages with pinned versions
!pip install transformers==4.51.3 torch==2.6.0 newspaper3k==0.2.8 nltk==3.9.1 lxml_html_clean==0.3.1



In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from newspaper import Article
from transformers import BartForConditionalGeneration, BartTokenizer

# Download NLTK data
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:

# Extractive summarization (NLTK)
def extractive_summary(text, num_sentences=3):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    word_freq = {word: words.count(word) for word in set(words) if word.isalnum() and word not in stop_words}

    sentences = sent_tokenize(text)
    sentence_scores = {sent: sum(word_freq.get(word, 0) for word in word_tokenize(sent.lower())) for sent in sentences}

    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    return ' '.join(summary_sentences)



In [11]:
# Abstractive summarization (BART)
def abstractive_summary(text):
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    inputs = tokenizer(text, max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, num_beams=4)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [12]:

# Main function
def quick_bit():
    print("QuickBit: News Summarizer")
    choice = input("Enter 1 for URL or 2 for text: ")

    text = ""
    if choice == '1':
        url = input("Enter news article URL: ")
        try:
            article = Article(url)
            article.download()
            article.parse()
            text = article.text
        except:
            print("Error fetching article. Try another URL or use text.")
            return
    elif choice == '2':
        text = input("Paste article text: ")
    else:
        print("Invalid choice. Use 1 or 2.")
        return

    if not text.strip():
        print("No text provided.")
        return

    print("\nExtractive Summary:")
    print(extractive_summary(text))
    print("\nAbstractive Summary:")
    print(abstractive_summary(text))

# Run


In [13]:
quick_bit()

QuickBit: News Summarizer
Enter 1 for URL or 2 for text: 1
Enter news article URL: https://www.thehindu.com/news/national/pahalgam-terror-attack-operation-sindoor-launch-live-updates-may-7-2025/article69543511.ece

Extractive Summary:
India exercised its right to act on terror, Foreign Secretary Vikram Misri said on the launch of military strikes on nine targets in Pakistan and Pakistan Occupied Jammu and Kashmir (POJK) under ‘Operation Sindoor’ while stressing that the actions were “measured, non-escalatory, proportionate, and responsible” and focused on dismantling the terrorist infrastructure and disabling terrorists likely to be sent across to India. Also read | Pakistan Army vows to respond to Indian air strikes

The strikes on the nine targets, four in Pakistan and five in POJK, were conducted between 01:05Am to 01:30AM on Wednesday. The Indian Armed Forces on Wednesday (May 7, 2025) launched ‘Operation Sindoor’, hitting terrorist infrastructure in Pakistan and Pakistan-occupied 